In [6]:
import numpy as np

def read_transportation_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        parameters['Number of Suppliers'] = int(lines[0].strip())
        parameters['Number of Customers'] = int(lines[1].strip())
        parameters['Supply Capacities'] = [int(x) for x in lines[2].strip().split()]
        parameters['Demand Requirements'] = [int(x) for x in lines[3].strip().split()]
        parameters['Cost Matrix'] = []
        for line in lines[4:]:
            parameters['Cost Matrix'].append([int(x) for x in line.strip().split()])
    return parameters

def northwest_corner(cost_matrix, supply, demand):
    supply_indices = np.arange(len(supply))
    demand_indices = np.arange(len(demand))
    allocation = np.zeros((len(supply), len(demand)))

    i = 0
    j = 0

    while i < len(supply) and j < len(demand):
        if supply[i] >= demand[j]:
            allocation[i][j] = demand[j]
            supply[i] -= demand[j]
            j += 1
        else:
            allocation[i][j] = supply[i]
            demand[j] -= supply[i]
            i += 1

    return allocation

file_path = '/Users/elyesskanderi/Documents/opti.txt'
transportation_parameters = read_transportation_parameters(file_path)

cost_matrix = np.array(transportation_parameters['Cost Matrix'])
supply = transportation_parameters['Supply Capacities']
demand = transportation_parameters['Demand Requirements']

initial_allocation = northwest_corner(cost_matrix, supply, demand)
print("Initial Feasible Solution (Northwest Corner Rule):")
print(initial_allocation)



Initial Feasible Solution (Northwest Corner Rule):
[[ 80.  20.   0.   0.]
 [  0. 100.  50.   0.]
 [  0.   0.  50. 150.]]


In [3]:
import numpy as np

def read_transportation_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        parameters['Number of Suppliers'] = int(lines[0].strip())
        parameters['Number of Customers'] = int(lines[1].strip())
        parameters['Supply Capacities'] = [int(x) for x in lines[2].strip().split()]
        parameters['Demand Requirements'] = [int(x) for x in lines[3].strip().split()]
        parameters['Cost Matrix'] = []
        for line in lines[4:]:
            parameters['Cost Matrix'].append([int(x) for x in line.strip().split()])
    return parameters

def minimum_cost_method(cost_matrix, supply, demand):
    allocation = np.zeros((len(supply), len(demand)))
    
    # Initialize remaining supply and demand
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()
    
    # Continue allocation until all supply or demand is exhausted
    while np.sum(remaining_supply) > 0 and np.sum(remaining_demand) > 0:
        # Find the minimum cost cell
        min_cost = np.inf
        min_i, min_j = -1, -1
        for i in range(len(supply)):
            for j in range(len(demand)):
                if cost_matrix[i][j] < min_cost and remaining_supply[i] > 0 and remaining_demand[j] > 0:
                    min_cost = cost_matrix[i][j]
                    min_i, min_j = i, j
        
        # Allocate shipments from the minimum cost cell
        allocation_quantity = min(remaining_supply[min_i], remaining_demand[min_j])
        allocation[min_i][min_j] = allocation_quantity
        
        # Update remaining supply and demand
        remaining_supply[min_i] -= allocation_quantity
        remaining_demand[min_j] -= allocation_quantity
        
    return allocation

file_path = '/Users/elyesskanderi/Documents/opti.txt'
transportation_parameters = read_transportation_parameters(file_path)

cost_matrix = np.array(transportation_parameters['Cost Matrix'])
supply = transportation_parameters['Supply Capacities']
demand = transportation_parameters['Demand Requirements']

initial_allocation = minimum_cost_method(cost_matrix, supply, demand)
print("Initial Feasible Solution (Minimum Cost Method):")
print(initial_allocation)


Initial Feasible Solution (Minimum Cost Method):
[[  0.   0. 100.   0.]
 [  0.   0.   0. 150.]
 [ 80. 120.   0.   0.]]


In [3]:
import numpy as np

def read_transportation_parameters(file_path):
    parameters = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        parameters['Number of Suppliers'] = int(lines[0].strip())
        parameters['Number of Customers'] = int(lines[1].strip())
        parameters['Supply Capacities'] = [int(x) for x in lines[2].strip().split()]
        parameters['Demand Requirements'] = [int(x) for x in lines[3].strip().split()]
        parameters['Cost Matrix'] = []
        for line in lines[4:]:
            parameters['Cost Matrix'].append([int(x) for x in line.strip().split()])
    return parameters

def minimum_row_cost_method(parameters):
    try:
        supply = parameters['Supply Capacities']
        demand = parameters['Demand Requirements']
        cost_matrix = parameters['Cost Matrix']

        # Initialize allocation matrix
        allocations = np.zeros((len(supply), len(demand)))

        # Iterate over rows
        for i in range(len(supply)):
            # Find the column index with the minimum cost
            min_col_index = np.argmin(cost_matrix[i])

            # Allocate units based on minimum of supply and demand
            quantity = min(supply[i], demand[min_col_index])
            allocations[i][min_col_index] = quantity
            supply[i] -= quantity
            demand[min_col_index] -= quantity

        return allocations
    
    except Exception as e:
        print("An error occurred:", e)
        return None

# Read transportation parameters from file
file_path = '/Users/elyesskanderi/Documents/opti.txt'
parameters = read_transportation_parameters(file_path)

# Minimum Row Cost Method
min_row_allocation = minimum_row_cost_method(parameters)
print("Minimum Row Cost Method Allocation:\n", min_row_allocation)


Minimum Row Cost Method Allocation:
 [[80.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [14]:
import numpy as np

def vogel_method(costs, supply, demand):
    num_sources = len(supply)
    num_destinations = len(demand)
    
    # Creating the matrix of remaining costs
    remaining_costs = np.array(costs)
    
    # Initializing variables to store excess and shortage
    supply_excess = np.array(supply)
    demand_shortage = np.array(demand)
    
    # Initializing variables to store indices of the largest penalty
    max_penalty_row = -1
    max_penalty_col = -1
    
    # Initializing the solution
    solution = np.zeros((num_sources, num_destinations))
    
    # While there is supply or demand remaining
    while np.any(supply_excess > 0) and np.any(demand_shortage > 0):
        max_penalty = -1
        
        # Computing shortages in rows and columns
        for i in range(num_sources):
            row_costs = np.sort(remaining_costs[i])
            penalty = row_costs[1] - row_costs[0]
            if penalty > max_penalty:
                max_penalty = penalty
                max_penalty_row = i
        
        for j in range(num_destinations):
            col_costs = np.sort(remaining_costs[:, j])
            penalty = col_costs[1] - col_costs[0]
            if penalty > max_penalty:
                max_penalty = penalty
                max_penalty_col = j
        
        # Identifying the largest shortage and assigning corresponding capacity
        if supply_excess[max_penalty_row] > demand_shortage[max_penalty_col]:
            solution[max_penalty_row, max_penalty_col] = demand_shortage[max_penalty_col]
            supply_excess[max_penalty_row] -= demand_shortage[max_penalty_col]
            demand_shortage[max_penalty_col] = 0
        else:
            solution[max_penalty_row, max_penalty_col] = supply_excess[max_penalty_row]
            demand_shortage[max_penalty_col] -= supply_excess[max_penalty_row]
            supply_excess[max_penalty_row] = 0
        
        # Updating remaining costs
        remaining_costs[max_penalty_row, :] = np.max(remaining_costs) + 1
        remaining_costs[:, max_penalty_col] = np.max(remaining_costs) + 1
    
    return solution

# Problem data
costs = [[10, 6, 9, 5],
         [8, 10, 7, 6],
         [5, 8, 9, 7]]

supply = [20, 30, 30]
demand = [25, 20, 25, 10]

# Solving the problem using Vogel's method
solution = vogel_method(costs, supply, demand)

print("Solution to the transportation problem using Vogel's method:")
print(solution)


Solution to the transportation problem using Vogel's method:
[[ 0. 20.  0.  0.]
 [ 0.  0. 20. 10.]
 [25.  0.  5.  0.]]


TRANSPORTATION SIMPLEX ALGORITHM

In [15]:
import numpy as np

def find_initial_basic_feasible_solution(supply, demand):
    num_suppliers = len(supply)
    num_customers = len(demand)
    initial_solution = np.zeros((num_suppliers, num_customers))
    for i in range(num_suppliers):
        for j in range(num_customers):
            quantity = min(supply[i], demand[j])
            initial_solution[i, j] = quantity
            supply[i] -= quantity
            demand[j] -= quantity
            if supply[i] == 0:
                break
    return initial_solution

def transportation_simplex_algorithm(cost_matrix, supply, demand):
    num_suppliers = len(supply)
    num_customers = len(demand)
    initial_solution = find_initial_basic_feasible_solution(supply, demand)
    if np.sum(initial_solution) != np.sum(supply) + np.sum(demand):
        raise ValueError("Initial solution is not feasible.")
    return initial_solution

cost_matrix = np.array([[3, 4, 6, 8], [5, 7, 9, 11], [2, 3, 4, 5]])
supply = np.array([100, 150, 200])
demand = np.array([80, 120, 100, 150])

initial_solution = find_initial_basic_feasible_solution(supply, demand)
print("Initial Basic Feasible Solution:")
print(initial_solution)

solution = transportation_simplex_algorithm(cost_matrix, supply, demand)


Initial Basic Feasible Solution:
[[ 80.  20.   0.   0.]
 [  0. 100.  50.   0.]
 [  0.   0.  50. 150.]]
